### **Данные**
**text_features.pq** - вектора текстов объявлений

**cat_features.pq** - фичи объявлений

**clickstream.pq** - история взаимодействия пользователей с объявлениями

**events.pq** - информация о типах событий

**submit_example.csv** - пример файла с предсказанием

**test_users.pq** - пользователи, для которых нужно отправить предсказания

**baseline.ipynb** -  бейзлайн как обучить модельку , разбить на валидацию и подготовить файл для отправки


### **Расшифровка**

**text_features.pq**

item  - id объявления [int64]

title_projection - вектор тайтла объявления array[int8, 64]. Чем выше скалярное произведение между 2-мя векторами, тем более похожи заголовки объявлений

**cat_features.pq**

item -  id объявления [int64]

location - id локации айтема.  [int64]

category - id категории товара. [int64]

clean_params - параметры объявления. В примере [{"attr":859,"value":653982} …]  attr - это id атрибута, value - это id значения атрибута [str]

node - id группы товара. Сущность , которую необходимо предсказать [int64]

**clickstream.pq**

cookie - id пользователя [int64]

item - id объявления [int64]

event - id типы события [int64]

event_date - время, когда пользователь провзаимодействовал с айтемом datetime[ns]

surface  - экран, с которого было взамодействие. Например пользователь может кликнуть на айтем с поисковой выдачи, во вкладке “Избранное” [int64]

platform -  тип устройства, с которого пользователь совершил действие. Например - android, ios, desktop, браузер в телефоне [int64]

node - id группы товара. Сущность , которую необходимо предсказать [int64]

**events.pq**

event - id типы события [int64]

is_contact - 0 или 1. Является ли событие контактным [int64]

**submit_example.csv**

cookie  - id пользователя [int64]

node - id группы товара. Сущность , которую необходимо предсказать [int64]

**test_users.pq**

cookie - id пользователя [int64]

# Load Data

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
!pip install polars


In [8]:
#!pip install implicit
!pip install /kaggle/input/d/emanonel/implicit/implicit-0.7.2-cp310-cp310-manylinux2014_x86_64.whl

Processing /kaggle/input/d/emanonel/implicit/implicit-0.7.2-cp310-cp310-manylinux2014_x86_64.whl


In [9]:
from datetime import timedelta
import polars as pl
import implicit

In [13]:
DATA_DIR = r'/kaggle/input/final-data'

df_test_users = pl.read_parquet(f'{DATA_DIR}/test_users.pq')
df_clickstream = pl.read_parquet(f'{DATA_DIR}/clickstream.pq')

df_cat_features = pl.read_parquet(f'{DATA_DIR}/cat_features.pq')
df_text_features = pl.read_parquet(f'{DATA_DIR}/text_features.pq')
df_event = pl.read_parquet(f'{DATA_DIR}/events.pq')

In [14]:
print(type(df_text_features))

<class 'polars.dataframe.frame.DataFrame'>


In [15]:
df_cat_features

item,location,category,clean_params,node
i64,i64,i64,str,u32
9,8385,57,"""[{""attr"":1157,""value"":664427},…",194747
17,2707,35,"""[{""attr"":2140,""value"":501466},…",352905
144,8383,8,"""[{""attr"":802,""value"":35791},{""…",17188
202,5397,57,"""[{""attr"":1157,""value"":490527},…",194766
236,2105,64,"""[{""attr"":112,""value"":420797},{…",153951
…,…,…,…,…
28804461,24,35,"""[{""attr"":2140,""value"":364348},…",326792
28804502,2305,51,"""[{""attr"":4622,""value"":171723},…",401208
28804563,2348,0,"""[{""attr"":914,""value"":93691},{""…",13974


# PREPARE TRAIN EVAL

Разделения данных на "старые" и "новые" на основе последней доступной даты в данных.

In [16]:
EVAL_DAYS_TRESHOLD = 14

In [17]:
treshhold = df_clickstream['event_date'].max() - timedelta(days=EVAL_DAYS_TRESHOLD)

В train попадают данные до пороговой даты, в eval после

In [18]:
df_train = df_clickstream.filter(df_clickstream['event_date']<= treshhold)
df_eval = df_clickstream.filter(df_clickstream['event_date']> treshhold)[['cookie', 'node', 'event']]

In [19]:
df_eval = df_eval.join(df_train, on=['cookie', 'node'], how='anti')


Anti join оставляет в df_eval только те строки, которых нет в df_train для указанных ключей (cookie и node).

In [20]:
df_eval = df_eval.filter(
    pl.col('event').is_in(
        df_event.filter(pl.col('is_contact')==1)['event'].unique()
    )
)

In [21]:
df_eval = df_eval.filter(
        pl.col('cookie').is_in(df_train['cookie'].unique())
    ).filter(
        pl.col('node').is_in(df_train['node'].unique())
    )

In [22]:
df_eval = df_eval.unique(['cookie', 'node'])

In [23]:
display(df_train.head(5), df_eval)

cookie,item,event,event_date,platform,surface,node
i64,i64,i64,datetime[ns],i64,i64,u32
0,19915558,17,2025-02-05 02:30:59,3,2,115659
0,2680232,17,2025-01-24 21:16:57,3,2,115829
1,4247649,17,2025-01-29 23:00:58,2,2,7
1,2171135,17,2025-01-17 19:23:29,2,2,214458
1,14233689,17,2025-01-16 19:36:23,2,2,214458


cookie,node,event
i64,u32,i64
60601,195256,4
32448,71536,0
21444,214244,10
18643,1923,15
84031,242794,10
…,…,…
104856,154001,10
149443,159211,10
8182,239953,5


# TRAIN MODEL

## ALS

In [26]:
def get_als_pred(users, nodes, user_to_pred):
    user_ids = users.unique().to_list()
    item_ids = nodes.unique().to_list()

    user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
    item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
    index_to_item_id = {v:k for k,v in item_id_to_index.items()}

    rows = users.replace_strict(user_id_to_index).to_list()
    cols = nodes.replace_strict(item_id_to_index).to_list()

    values = [1] * len(users)

    sparse_matrix = csr_matrix((values, (rows, cols)), shape=(len(user_ids), len(item_ids)))

    model = implicit.als.AlternatingLeastSquares(iterations=10, factors=60)
    model.fit(sparse_matrix, )


    user4pred = np.array([user_id_to_index[i] for i in user_to_pred])

    recommendations, scores = model.recommend(user4pred, sparse_matrix[user4pred], N=40, filter_already_liked_items=True)

    df_pred = pl.DataFrame(
        {
            'node': [
                [index_to_item_id[i] for i in i] for i in recommendations.tolist()
            ],
             'cookie': list(user_to_pred),
            'scores': scores.tolist()

        }
    )
    df_pred = df_pred.explode(['node', 'scores'])
    return df_pred

In [27]:
from scipy.sparse import csr_matrix
import numpy as np
import implicit


users = df_train["cookie"]
nodes = df_train["node"]
eval_users = df_eval['cookie'].unique().to_list()

df_pred = get_als_pred(users, nodes,eval_users )




  (0, 1789)	1
  (0, 1793)	1
  (0, 1796)	1
  (0, 1806)	8
  (0, 46642)	4
  (0, 46644)	1
  (0, 63189)	1
  (0, 70044)	11
  (0, 70046)	2
  (0, 70047)	2
  (0, 70090)	1
  (0, 103483)	6
  (0, 103484)	1
  (0, 103492)	14
  (0, 103508)	1
  (0, 103513)	2
  (0, 103561)	19
  (0, 103562)	4
  (0, 103631)	3
  (0, 103632)	4
  (0, 103640)	4
  (0, 103641)	1
  (0, 103642)	3
  (0, 103647)	2
  (0, 103649)	16
  :	:
  (126792, 208918)	2
  (126792, 209700)	1
  (126792, 209706)	2
  (126792, 209711)	1
  (126792, 209712)	6
  (126792, 210207)	2
  (126792, 210208)	1
  (126792, 210209)	2
  (126792, 210246)	2
  (126792, 210247)	1
  (126792, 210266)	1
  (126792, 210883)	2
  (126792, 210886)	1
  (126792, 216275)	1
  (126792, 217113)	1
  (126792, 217124)	1
  (126792, 217591)	1
  (126792, 221068)	3
  (126792, 221070)	2
  (126792, 221092)	4
  (126792, 226476)	1
  (126792, 229027)	1
  (126792, 283900)	1
  (126792, 334310)	1
  (126792, 378385)	1


/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()
/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/10 [00:00<?, ?it/s]

## popular

In [28]:
def get_popular(df):
    popukar_node = df.group_by('node').agg(pl.col('cookie').count()).sort('cookie').tail(40)['node'].to_list()
    df_pred_pop = pl.DataFrame({'node': [popukar_node for i in range(len(eval_users))], 'cookie': eval_users})
    df_pred_pop = df_pred_pop.explode('node')
    return df_pred_pop

train_pop = get_popular(df_train)


# CALC EVAL METRICS

In [31]:
def recall_at(df_true, df_pred, k=40):
    return df_true[['node', 'cookie']].with_columns(
        pl.col('node').cast(pl.Int64)  # Приводим к i64 (как в df_pred)
    ).join(
        df_pred.group_by('cookie').head(k).with_columns(
            value=1,
            node=pl.col('node').cast(pl.Int64)  # На всякий случай тоже приводим
        )[['node', 'cookie', 'value']],
        how='left',
        on=['cookie', 'node']
    ).select(
        [pl.col('value').fill_null(0), 'cookie']
    ).group_by(
        'cookie'
    ).agg(
        (pl.col('value').sum() / pl.col('value').count()).alias('recall')
    )['recall'].mean()

In [32]:
recall_at(df_eval, df_pred, k=40)

0.15238566196115227

In [33]:
recall_at(df_eval, train_pop, k=40)

0.058067308552970216

# SUMBIT

In [ ]:
users = df_clickstream["cookie"]
nodes = df_clickstream["node"]
test_users = df_test_users['cookie'].unique().to_list()

df_pred = get_als_pred(users, nodes, test_users )


In [ ]:
df_pred.write_csv('prediction.csv')